# UFF DeepSeek 6.7B Training - Tank Building Methodology

Training UFF (UFM Federation Framework) model using Tank Building methodology:
- **Model**: DeepSeek 6.7B-FP16
- **Training**: RL + Supervised Fine-tuning
- **Supervision**: Claude Code architectural guidance
- **GPU**: Kaggle P100 (16GB VRAM)
- **Time Limit**: 12 hours (within 20h/week)


In [ ]:
# Install dependencies
!pip install torch transformers deepspeed accelerate wandb
!pip install datasets tokenizers
!nvidia-smi  # Check GPU availability

In [ ]:
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import deepspeed
from datetime import datetime

print(f"🚀 Starting UFF DeepSeek training at {datetime.now()}")
print(f"🔥 GPU Available: {torch.cuda.is_available()}")
print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Load Tank Building training data
with open('/kaggle/input/uff-tank-building-corpus/training_data.json', 'r') as f:
    training_data = json.load(f)

print(f"📊 Loaded {len(training_data)} Tank Building sessions")
print(f"📋 Stages represented: {set(session.get('tank_building_stage') for session in training_data)}")

In [ ]:
# Initialize DeepSeek model and tokenizer
model_name = "deepseek-ai/deepseek-coder-6.7b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print(f"✅ Model loaded: {model_name}")
print(f"📏 Model parameters: {sum(p.numel() for p in model.parameters()) / 1e9:.1f}B")

In [ ]:
# Initialize DeepSpeed training
with open('/kaggle/input/uff-tank-building-corpus/deepspeed_config.json', 'r') as f:
    ds_config = json.load(f)

model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    config=ds_config
)

print("⚡ DeepSpeed initialized for UFF training")
print(f"🎯 Batch size: {ds_config['train_micro_batch_size_per_gpu']}")
print(f"🔄 Gradient accumulation: {ds_config['gradient_accumulation_steps']}")

In [ ]:
# Training loop with Tank Building methodology
import time

def train_uff_model(training_data, model_engine, tokenizer, max_hours=12):
    """Train UFF model with Tank Building methodology"""
    
    start_time = time.time()
    max_seconds = max_hours * 3600
    
    step = 0
    total_loss = 0
    
    print(f"🏗️  Starting Tank Building training for {max_hours} hours")
    
    for epoch in range(100):  # Large number, will be limited by time
        if time.time() - start_time > max_seconds:
            print(f"⏰ Reached {max_hours} hour limit, stopping training")
            break
            
        for session in training_data:
            if time.time() - start_time > max_seconds:
                break
                
            # Process Tank Building session
            stage = session.get('tank_building_stage', 'stage_1')
            component_code = session.get('component_code', '')
            claude_feedback = session.get('claude_feedback', [])
            
            # Create training prompt
            prompt = f"Tank Building {stage}: {component_code}"
            
            # Tokenize and train
            inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(model_engine.device) for k, v in inputs.items()}
            
            outputs = model_engine(**inputs, labels=inputs['input_ids'])
            loss = outputs.loss
            
            model_engine.backward(loss)
            model_engine.step()
            
            total_loss += loss.item()
            step += 1
            
            # Log progress every 100 steps
            if step % 100 == 0:
                avg_loss = total_loss / step
                elapsed = (time.time() - start_time) / 3600
                print(f"Step {step} | Loss: {avg_loss:.4f} | Elapsed: {elapsed:.2f}h | Stage: {stage}")
                
    return step, total_loss / step

# Run training
steps, avg_loss = train_uff_model(training_data, model_engine, tokenizer, max_hours=12)
print(f"🎉 Training completed! Steps: {steps}, Average Loss: {avg_loss:.4f}")

In [ ]:
# Save trained model
output_dir = "./uff_deepseek_trained"
model_engine.save_checkpoint(output_dir)

# Also save in HuggingFace format for easy deployment
model.save_pretrained("./uff_model_hf")
tokenizer.save_pretrained("./uff_model_hf")

print("💾 Model saved successfully!")
print(f"📁 DeepSpeed checkpoint: {output_dir}")
print(f"📁 HuggingFace format: ./uff_model_hf")

In [ ]:
# Test component generation
def test_uff_generation(prompt, max_length=200):
    """Test UFF model component generation"""
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test Tank Building stages
test_prompts = [
    "Tank Building stage_1: Create a simple file reader component",
    "Tank Building stage_2: Extend file reader with error handling", 
    "Tank Building stage_3: Optimize file reader for production",
    "Tank Building stage_4: Add advanced features to file reader"
]

print("🧪 Testing UFF component generation:")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Test {i} ---")
    print(f"Prompt: {prompt}")
    result = test_uff_generation(prompt)
    print(f"Generated: {result[len(prompt):]}")

In [ ]:
# Create training report
training_report = {
    "model": "UFF DeepSeek 6.7B-FP16",
    "methodology": "Tank Building + Claude Supervision",
    "training_steps": steps,
    "average_loss": avg_loss,
    "training_time_hours": 12,
    "gpu": "Kaggle P100 16GB",
    "dataset_size": len(training_data),
    "timestamp": datetime.now().isoformat()
}

with open("uff_training_report.json", "w") as f:
    json.dump(training_report, f, indent=2)
    
print("📊 Training report saved!")
print(json.dumps(training_report, indent=2))